# Implementing RSA from scratch

In [1]:
import random

## ASCII encoding schemes
Via `ord`, a string is just a sequence of bytes, which we can think of as a number in base 256. Therefore translating a string to an integer $n$ and back should can be thought of as computing the number from this base 256 representation.

`textToInt` takes a string w as input: a list of characters $c_0c_1c_2c_3...c_r$
and outputs an integer $n$ satisfying: $$n=\sum_{i=0}^{r} ord(c_i)*256^i$$

In [2]:
def textToInt(words):
    number = 0
    i = 0
    for letter in words:
        number += ord(letter)*(256**i)
        i+=1
    return number

`intToText` does the inverse job: finding the base 256 expansion $n$, and then converting each *'digit'* (which is a byte, i.e., a number 0,...,255) into its corresponding letter using `chr`:

In [3]:
def intToText(number):
    words = ""
    while number>0:
        nextLetter = number % 256
        words += chr(nextLetter)
        number = (number-nextLetter)//256
    return words

## Maths functions
Here some useful maths tools are defined. They will be used throughout the whole notebook.

`divisionWithRemainder` takes two integers and returns the result of the integer division between them and the remainder

In [4]:
def divisionWithRemainder(a,b):
    return [a//b, a%b]

`findGCDFast` takes two integers $a$ and $b$ and returns their GCD. It implements the Euclidean algorithm and uses the `divisionWithRemainder` function

`extendedGCD` works with the same philosophy but it is optimized and returns the GCD and the integers $u$ and $v$ such that: $$au + bv = GCD(a,b)$$

In [5]:
def findGCDFast(a,b):
    q, r = divisionWithRemainder(a,b)
    while r > 0:
        a = b
        b = r
        q, r = divisionWithRemainder(a,b)
    return b

def extendedGCD(a,b):
    u = 1
    g = a
    x = 0
    y = b
    while y > 0:
        q,t = divisionWithRemainder(g,y)
        s = u - q*x
        u, g = x, y
        x, y = s, t
    v = (g - a*u)//b
    return [g,u,v]

In [6]:
extendedGCD(543,324)

[3, 37, -62]

The fast powering algorithm takes as input natural numbers $N, A$,
and an element $g \in \mathbb{Z}/N\mathbb{Z}$, and returns $g^A \ (mod \ N)$, that is, the reduction of $g^A$ modulo $N$. It requires computing the binary expansion of the number $A$.
For large values of $A$, it requires a remembering large amounts of data,
which is not ideal. Here `fastPowerSmall` is a variant of the fast powering algorithm which avoids the need for storage.

In [7]:
def fastPowerSmall(g,A,N):
    a = g
    b = 1
    while A>0:
        if A % 2 == 1:
            b = b * a % N
        A = A//2
        a = a*a % N
    return b

The problem at the center of RSA is finding the e’th root of the ciphertext $c$ modulo $N$ where $N = pq$ is a product of (distinct) primes. This is hard to do for large $N$ if you don’t know the factors $p$ and $q$, but once you know them, it becomes easy!

`findRoot` takes as input 4 integers:
- $c$: the ciphertext
- $e$: the encryption exponent
- $p,q$: the two prime factors of $N$

and solves the equation $x^e \equiv c\ \ (mod N)$ for $x$ where $N = pq$.

The function uses some mathematical results and the functions we defined before.

In [8]:
def findRoot(c,e,p,q):
    if extendedGCD(e,(p-1)*(q-1))[0] == 1:
        g = findGCDFast(p-1,q-1)
        m = (p-1)*(q-1)/g
        d = extendedGCD(e,m)[1] % m
        return fastPowerSmall(c,d,p*q)
    else:
        print("Error: e and (p-1)*(q-1) must be coprimes")

In [9]:
findRoot(43927,17389,229,281)

14458

In [10]:
findRoot(134872,9843,104729,287117)

25470280263

## Finding primes
The security of RSA depends on finding two large primes $p$ and $q$, and multiplying them together to get a large value $N$. As said before, knowing $p$ and $q$ makes the RSA problem easy, so it is important that you keep $p$ and $q$ secret. The best way to do this is to generate them yourself, using the following tools.

Firstly, we need to implement a primality test: here a probabilistic test based on *Miller-Rabin witnesses* is used. 

**Definition.** Let n be an odd number and write $n-1=2^{k}q$ with $q$ odd. An
integer $a$ satisfying $GCD(a, n) = 1$ is called a *Miller–Rabin witness for (the compositeness of)* $n$ if both of the following conditions are true:
- $a^q \neq 1\ \ (mod\ n)$
- None of $a^{2^{i}q} \equiv -1\ \ (mod\ n)$ for all $i = 0, 1, 2,...,k - 1$

It can be proven that if there exists a *Miller–Rabin witness for* $n$, then $n$ is definitely a composite number.

The function `millerRabin` takes as input integers $a, n \in \mathbb{Z}$ with $2 ≤ a ≤ n-1$ and returns `True` if $a$ is a *Miller-Rabin witness for (the compositeness of)* $n$ and `False` otherwise.

The function `pow2_times_odd` is just an utility function used in the `millerRabin` function.

Note that using the fast powering function `fastPowerSmall` in `millerRabin` is crucial because without it, everything would slow down significantly as the numbers get bigger and bigger.

In [11]:
def pow2_times_odd(m):
    k = 0
    while m%2 == 0:
        k += 1
        m //= 2
    return k,m

def millerRabin(a, n):
    if (n%2 == 0 or 1 < findGCDFast(a,n) < n):
        return True
    k = pow2_times_odd(n-1)[0]
    q = pow2_times_odd(n-1)[1]
    a = fastPowerSmall(a,q,n)
        
    if a % n == 1:
        return False
        
    for i in range(k):
        if a % n == n-1:
            return False
        a = a*a % n
    return True

Finding one *Miller-Rabin witness* $a$ means that $n$ is composite, but if $a$ is not a *Miller-Rabin witness*, that does not mean that $n$ is prime. Nevertheless, *Miller-Rabin witnesses* are very plentiful for composite numbers. We exploit this to write a probabilistic test for primality.

`probablyPrime` runs `millerRabin` for 20 randomly chosen $a$ with $2 ≤ a ≤ n-1$. If any of these $a$ are a *witness*, then $n$ must be composite,
so it return `False`. Otherwise, it is very likely that $n$ is prime, so we return `True`.

In [12]:
def probablyPrime(n):
    for i in range(20):
        a = random.randrange(2, n)
        if millerRabin(int(a),n) == True:
            return False
    return True

So far, we have built a way to efficiently test if a given number is (very likely) prime. Now, how to find primes in a given range? It turns out that primes are plentiful enough that if we just guess random numbers and see if they are prime, we will probably get one before too long.

`findPrime` returns a prime between the two bounds by repeatedly picking a random number $n$ between the bounds and running `probablyPrime`, until it finds an $n$ that is (probably) prime, and returns it.

In [13]:
def findPrime(lowerBound, upperBound):
    n = random.randrange(lowerBound, upperBound)
    while not probablyPrime(n):
        n = random.randrange(lowerBound, upperBound)
        if probablyPrime(n) == True:
            return n
    return n

Let's try if everything works finding a prime with 100 digits. We check if our number is actually a prime using the `sympy` library.

In [15]:
%pip install sympy

Defaulting to user installation because normal site-packages is not writeable
  Using cached sympy-1.11.1-py3-none-any.whl (6.5 MB)
  Using cached mpmath-1.2.1-py3-none-any.whl (532 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [16]:
prime = findPrime(int(1e99),int(1e100-1))
print(prime)
print()

import sympy
print(sympy.isprime(prime))

5721294443540187064776952751611002161800998286310512397841933349242177528112372632396842958698873147

True


## RSA in its full glory
Now we have all the tools needed to implement the RSA cryptography. To do so, we have to create the three main RSA building blocks:
- An RSA key generator
- An encryption function
- A decryption function

To generate the key we define a function `generateRSAKey` which takes as input an integer $b$ and generates an RSA public and private key from primes $b$ bits long in the following four steps:

1. Generate 2 primes $p$ and $q$ of length $b$ bits using the functions defined before.
2. Choose an encryption exponent $e \in (\mathbb{Z}/(p - 1)(q - 1) \mathbb{Z})^*$ with $e \neq 1$.
3. Compute the decryption exponent $d$ associated to $e$.
4. Return a pair of keys `[PublicKey,PrivateKey]` where:
    - `PublicKey = [N,e]`  will be published; ($N = pq$)
    - `PrivateKey = [N,d]` will be kept secret and used to decrypt messages.

In [17]:
def generateRSAKey(b):
    p = findPrime(2**(b-1),2**b - 1)
    q = findPrime(2**(b-1),2**b - 1)
    
    e = 2
    while findGCDFast(e, (p-1)*(q-1)) != 1:
        e = random.randrange(2, (p-1)*(q-1) - 1)
        
    g = findGCDFast(p-1,q-1)
    m = (p-1)*(q-1)//g
    d = extendedGCD(e,m)[1] % m
    
    PublicKey = [p*q, e]
    PrivateKey = [p*q, d]
    
    return [PublicKey, PrivateKey]

The encryption/decryption functions are pretty straightforward. See the table below to better understand what they do.

Note again the use of `fastPowerSmall` to speed up computations.

In [18]:
def RSAEncrypt(message, PublicKey):
    return fastPowerSmall(message,PublicKey[1],PublicKey[0])

def RSADecrypt(cipher, PrivateKey):
    return fastPowerSmall(cipher,PrivateKey[1],PrivateKey[0])

This schematic table clearly shows how RSA encryption/decryption works.
![alt text](rsa.PNG "RSA")
*Image credits: An Introduction to Mathematical Cryptography - Jeffrey Hoffstein, Jill Pipher, Joseph H. Silverman*

## Test: 16bit-RSA
Let's test our work with a small numeric message.

In [19]:
key = generateRSAKey(16)

In [20]:
message = 314159

In [21]:
cipher = RSAEncrypt(message, key[0])

In [22]:
RSADecrypt(cipher, key[1])

314159

The message got after the decryption process is the same of the initial one, so everything worked!

## Test: 512bit-RSA
Now, let's try with a longer text message. Clearly, we need more bit to store more information and that's why we are using 512 bit here.

In [27]:
key = generateRSAKey(512)
key[0]

[93791951647170464720011059182837961431056512215466403269451362758730863000893709630106951893038968907315335182167090205816550695958485215801115680290162432380563017834815012367411965369149929979671989831537837339644255830758483576838351704857357822394111131369152508750784448555107488401813792867872080144259,
 51154987670896938483485888594612962432206454212630477226545215180604556390235881159152221682902695868553853616422212728955235339700911561085289344818239411305908037142467440354528555609294923702043082686003745030723472443545753219109136633239278698887083425229579762117175639056867412722780355181431314602799]

In [24]:
message = "Hello, this is a test!"

In [25]:
cipher = RSAEncrypt(textToInt(message), key[0]) 

In [26]:
intToText(RSADecrypt(cipher, key[1]))

'Hello, this is a test!'

Again, the message got after the decryption process is the same of the initial one, so everything worked!